前两个notebook在简易数据上分别讲了基于用户与物品的CF算法，注意到两者都需要一个用户-物品矩阵，

In [1]:
import pandas as pd
import numpy as np
train = pd.read_table('../datasets/ml-100k/u1.base',
                      sep='\t', header=None).iloc[:, :3].values
test = pd.read_table('../datasets/ml-100k/u1.test',
                     sep='\t', header=None).iloc[:, :3].values
n_users, n_items = 943+1, 1682+1    # 数据idx从1开始
n_samples = train.shape[0]

print(train.shape, test.shape)

(80000, 3) (20000, 3)


In [2]:
np.max(train,axis=0)

array([ 943, 1682,    5], dtype=int64)

In [3]:
k = 20    # 隐因子数量

glob_mean = np.mean(train[:, 2])    # 全局均分

bi = np.zeros(n_items)
bu = np.zeros(n_users)
qi = np.zeros((n_items, k))
pu = np.zeros((n_users, k))

# 查询用字典，避免生成大型稀疏矩阵
item_user_dict = dict()
user_item_dict = dict()

In [4]:
for sample in train:
    user_id,item_id,rating=sample
    item_user_dict.setdefault(item_id,{})
    user_item_dict.setdefault(user_id,{})
    
    item_user_dict[item_id][user_id]=rating
    user_item_dict[user_id][item_id]=rating

In [11]:
max_iter = 20
lr = 0.01
alpha = 0.1

for epoch in range(max_iter):
    MSE = 0
    random_idxs = np.random.permutation(n_samples)

    for idx in random_idxs:
        user_id, item_id, rating = train[idx]
        y_pred = glob_mean+bi[item_id]+bu[user_id] + \
            np.dot(qi[item_id], pu[user_id].T)
        err = rating-y_pred
        MSE += err**2

        bu[user_id] += lr*(err-alpha*bu[user_id])
        bi[item_id] += lr*(err-alpha*bi[item_id])
        tmp = qi[item_id]
        qi[item_id] += lr*(err*pu[user_id]-alpha*qi[item_id])
        pu[user_id] += lr*(err*tmp-alpha*pu[user_id])

    MSE /= n_samples
    print(epoch,MSE)

0 0.8434991455144287
1 0.8436565999913066
2 0.8436005213010472
3 0.8436614845061078
4 0.8433439570864748
5 0.8434972913557541
6 0.8432513077754996
7 0.843459197069637
8 0.8431670911531315
9 0.8435748585503104
10 0.8433492386487466
11 0.8431863576950571
12 0.8434382732949214
13 0.8431897445144341
14 0.8431080678343126
15 0.843247019463502
16 0.8430651417595831
17 0.8431369740246276
18 0.8428642142854341
19 0.8431213424913869


In [12]:
Y_pred=list()
test_mse=0
for sample in test:
    user_id, item_id, rating = sample
    y_pred=glob_mean+bi[item_id]+bu[user_id] + \
            np.dot(qi[item_id], pu[user_id].T)
    test_mse+=(rating-y_pred)**2
test_mse/=len(test)
    
print(test_mse)

0.9139544401844456
